In [1]:
import cvxpy as cp
import pickle
import pandas as pd
import numpy as np

In [2]:
# config=(6,2)
config=(36, 5)
topo='ddf'
all_shortest_paths = dict(pickle.load(open(f'../pickled_data/graphs_and_paths/all_shortest_paths_{config}_{topo}_paths.pickle', 'rb')))
edge_list = pickle.load(open(f'../pickled_data/graphs_and_paths/{config}_{topo}_edgelist.pickle', 'rb'))

In [3]:
dict = {'source':[ ],
        'dest':[ ],
        'path':[ ],
        'edges': [] }

df = pd.DataFrame(dict)
for (u, v), paths in all_shortest_paths.items():
    for path in paths:
        new_data = {
            'source': int(u),
            'dest': int(v),
            'path': [path],
            'edges': [[(path[i], path[i+1]) for i in range(len(path)-1)]]
        }
        new_row = pd.DataFrame(new_data)
        df = pd.concat([df, new_row], ignore_index=True)  # Reset index using ignore_index=True


In [4]:
display(df)

,source,dest,path,edges
0,0.0,1.0,"[0, 1]","[(0, 1)]"
1,0.0,2.0,"[0, 2]","[(0, 2)]"
2,0.0,3.0,"[0, 3]","[(0, 3)]"
3,0.0,4.0,"[0, 4]","[(0, 4)]"
4,0.0,5.0,"[0, 4, 5]","[(0, 4), (4, 5)]"
...,...,...,...,...
1939,35.0,30.0,"[35, 31, 30]","[(35, 31), (31, 30)]"
1940,35.0,31.0,"[35, 31]","[(35, 31)]"
1941,35.0,32.0,"[35, 32]","[(35, 32)]"
1942,35.0,33.0,"[35, 33]","[(35, 33)]"


In [5]:
num_paths=len(df.index)
x=cp.Variable(num_paths)
print("number of paths:", num_paths)

number of paths: 1944


In [6]:
A=[]
for (u, v) in all_shortest_paths.keys():
    Aij=[0]*num_paths
    in_the_zoon=False
    for i in range(num_paths):
        if df.iloc[i]['source']==u and df.iloc[i]['dest']==v:
            Aij[i]=1
            in_the_zoon=True
        elif in_the_zoon == True:
            break
    
    A.append(Aij)

A=np.array(A)
# A=A.transpose()
constraints=[ A @ x == 1, x>=0, x<=1] 

In [7]:
# Define objective function
list_of_link_loads=[]
for (u,v) in edge_list:

    # one direction
    B=[0]*num_paths
    for i in range(num_paths):
        if (u,v) in df.iloc[i]['edges']:
            B[i]=1
    list_of_link_loads.append(B@x)

    # another direction
    B=[0]*num_paths
    for i in range(num_paths):
        if (v,u) in df.iloc[i]['edges']:
            B[i]=1
    list_of_link_loads.append(B@x)

list_of_link_loads=cp.hstack(list_of_link_loads)
objective = cp.Minimize(cp.max(list_of_link_loads))

In [14]:
# Check the available solvers
print(cp.installed_solvers())

['ECOS', 'ECOS_BB', 'OSQP', 'SCIPY', 'SCS']


In [19]:
problem = cp.Problem(objective, constraints)
problem.solve(solver=cp.SCIPY)

all_weighted_paths={}

if problem.status not in ["infeasible", "unbounded"]:
    # Otherwise, problem.value is inf or -inf, respectively.
    print("Optimal value: %s" % problem.value)
    for i, variable in enumerate(problem.variables()[0]):
        print("Variable %s: value %s" % (variable.name(), variable.value))
        source=df.iloc[i]['source']
        dest=df.iloc[i]['dest']
        if not( (source, dest) in all_weighted_paths.keys() ):
            all_weighted_paths[(source, dest)]=[]
        
        all_weighted_paths[(source, dest)].append( (df.iloc[i]['path'], variable.value) )
        
else:
    print(problem.status)

Optimal value: 17.571428571428704
Variable var1[0]: value 1.0
Variable var1[1]: value 1.0
Variable var1[2]: value 1.0
Variable var1[3]: value 1.0
Variable var1[4]: value 1.0
Variable var1[5]: value 1.0
Variable var1[6]: value 1.0
Variable var1[7]: value 1.0
Variable var1[8]: value 1.0
Variable var1[9]: value -0.0
Variable var1[10]: value 1.0
Variable var1[11]: value 1.0
Variable var1[12]: value 1.0
Variable var1[13]: value 1.0
Variable var1[14]: value -0.0
Variable var1[15]: value 1.0
Variable var1[16]: value 1.0
Variable var1[17]: value 1.0
Variable var1[18]: value -0.0
Variable var1[19]: value -0.0
Variable var1[20]: value 1.0
Variable var1[21]: value -0.0
Variable var1[22]: value -0.0
Variable var1[23]: value -0.0
Variable var1[24]: value 1.0
Variable var1[25]: value -0.0
Variable var1[26]: value 1.0
Variable var1[27]: value -0.0
Variable var1[28]: value -0.0
Variable var1[29]: value 1.0
Variable var1[30]: value 0.42857142857128605
Variable var1[31]: value 0.571428571428714
Variable

In [20]:
pickle.dump(all_weighted_paths, open(f'../pickled_data/graphs_and_paths/all_shortest_paths_LP_weighted_{config}_{topo}_paths.pickle', 'wb'))
